# Diluted Unitary Fitting

In [3]:
import sys
sys.path.insert(0, '../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle

from qiskit.quantum_info import DensityMatrix, random_unitary
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm
from math import ceil

from loss_functions import *
from optimization import *
from quantum_channel import *
from kraus_channels import *
from quantum_tools import *
from experimental import *
from spam import *
from scipy.stats import gaussian_kde
from quantum_circuits import *
from analysis import *
from utils import *

#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=4)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [4]:
def optimize_annulus_distance(spectrum_target = None, 
                              d = None, 
                              rank_start = 1, 
                              rank_step = 1,
                              num_iter = 1000,
                              lr = 0.001): 
    
    model_list = []
    distance_list = []

    for rank in range(rank_start, d**2+rank_step, rank_step): 
        if len(model_list)>0:
            c = model_list[0].channel.c
        else:
            c = 0.5
        model = ModelQuantumMap(channel = DilutedKrausMap(U=generate_haar_random(d, rng=rng), 
                                                          c=c, 
                                                          kraus_part=KrausMap(d, 
                                                                              rank=rank, 
                                                                              trainable=False),
                                                     ),
                            loss_function = AnnulusDistance(),
                            optimizer = tf.optimizers.Adam(learning_rate=lr),
                            logger = Logger(loss_function = AnnulusDistance(), 
                                            verbose=True,),
                            )
        model.train(inputs = None,
                    targets = [spectrum_target],
                    num_iter = num_iter,
                    
                   )

        model_list.append(model)
        distance_list.append(model.logger.loss_train_list[-1])
        idx = np.argmin(distance_list)
        
        if distance_list[idx]<distance_list[-1]:
            print("Done!")
            break
            
    
    idx = np.argmin(distance_list)
    distance_best = distance_list[idx]
    model_best = model_list[idx]

    return distance_best, model_best

def fit_diluted_unitary(spectrum_target_list, 
                        d, 
                        rank_start, 
                        rank_step,
                        num_iter = 1000,
                        lr = 0.001):
    
    distance_best_list = []
    model_best_list = []
    for spectrum_target in tqdm(spectrum_target_list):
        U = generate_haar_random(d, rng) 
        distance_best, model_best = optimize_annulus_distance(spectrum_target,
                                                              d,
                                                              rank_start,
                                                              rank_step,
                                                              num_iter = num_iter,
                                                              lr = lr)
        distance_best_list.append(distance_best)
        model_best_list.append(model_best)
    
    for model in model_best_list:
        model.optimizer = None
        
    return distance_best_list, model_best_list

## Three Qubits

In [8]:
[channel_list1, channel_list2, channel_list3, channel_list4, channel_list5, channel_list6, channel_list7] = loader(data_path("channel_fullRank_belem_longTrain.p"))

spectrum_target_list1 = [channel_spectrum(channel) for channel in channel_list1]
spectrum_target_list2 = [channel_spectrum(channel) for channel in channel_list2]
spectrum_target_list3 = [channel_spectrum(channel) for channel in channel_list3]
spectrum_target_list4 = [channel_spectrum(channel) for channel in channel_list4]
spectrum_target_list5 = [channel_spectrum(channel) for channel in channel_list5]
spectrum_target_list6 = [channel_spectrum(channel) for channel in channel_list6]
spectrum_target_list7 = [channel_spectrum(channel) for channel in channel_list7]

In [10]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)
rng = np.random.default_rng(seed=42)

distance_best_list, model_best_list = fit_diluted_unitary(spectrum_target_list1, 
                                                          d=d, 
                                                          rank_start=1, 
                                                          rank_step=1,
                                                          num_iter = 1000,
                                                         )

saver([distance_best_list, model_best_list], data_path("diluted_fitting_8layer.model"))

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

0.3002943596574898 None
0.24838912812754552 None
0.20478259098322216 None
0.18368825687693538 None
0.1375170540985229 None
0.1258290534983146 None
0.08483153144384839 None
0.06148836114199421 None
0.03935813165678399 None
0.037905488159005726 None
0.037913458302756164 None


  0%|          | 0/1000 [00:00<?, ?it/s]

0.02736806415366756 None
0.012214088101542968 None
0.012138353514836202 None
0.012162329146436014 None
0.028468315516770037 None
0.01214541126551746 None
0.012136968966235406 None
0.012165074337436351 None
0.012137535360292172 None
0.02848004963156246 None
0.012150703684341325 None


...

0.0018224011971919273 None
0.0009768025897201366 None
0.0009241024207237205 None
0.0009228821394939627 None
0.0009480233471613363 None
0.0009034119102037495 None
0.0008960967103166318 None
0.0009247063252797956 None
0.0009116872274688853 None
0.0009445772397152932 None
0.0009489671584236139 None


  0%|          | 0/1000 [00:00<?, ?it/s]

0.0013417626657505378 None
0.0008058520594482763 None
0.0007688746034733126 None
0.000855250581248142 None
0.0007696862020765869 None
0.0007449281863304842 None
0.0007661787786384415 None
0.0007402802992693423 None
0.000756287639986312 None
0.0007480041673849423 None
0.0007589109942692279 None


  0%|          | 0/1000 [00:00<?, ?it/s]

0.005775727658946176 None
0.0037972930616154317 None
0.0038692445533752414 None
0.0037792082957726616 None
0.0037979358083098575 None
0.003789078081478514 None
0.003792273129106325 None
0.0037961999704752163 None
0.0038027321600876203 None
0.00379441614487589 None
0.003814926946226933 None
Done!


In [11]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)
rng = np.random.default_rng(seed=42)

[distance_best_list, model_best_list] = loader(data_path("diluted_fitting_8layer.model"))
rank_start = model_best_list[-1].channel.rank

distance_best_list, model_best_list = fit_diluted_unitary(spectrum_target_list2, 
                                                          d=d, 
                                                          rank_start=rank_start, 
                                                          rank_step=1,
                                                          num_iter = 1000)

saver([distance_best_list, model_best_list], data_path("diluted_fitting_16layer.model"))

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

0.020721186801084628 None
0.004716536950984276 None
0.004687704153113991 None
0.004698283086504492 None
0.004677117888232848 None
0.004709187388755473 None
0.004670030845179595 None
0.0046728034876462165 None
0.004698958987577027 None
0.00467597439346852 None
0.004708099780371658 None


  0%|          | 0/1000 [00:00<?, ?it/s]

0.004778098115815989 None
0.0026184824309006295 None
0.002584676876820174 None
0.0025874275552148986 None
0.0026101016918631515 None
0.0026227251230063454 None
0.0025914915223684775 None
0.0025860081296860756 None
0.0026234729906034885 None
0.0026044742182862054 None
0.002608190302301437 None


...

0.009069626026052305 None
0.0034003976338337016 None
0.003365020521630985 None
0.0033443238646641738 None
0.0033555888128811136 None
0.0033813837923908174 None
0.0033710454116681926 None
0.0033291871506053823 None
0.0033331451718793333 None
0.003365818212677538 None
0.003350401200955169 None


  0%|          | 0/1000 [00:00<?, ?it/s]

0.004087271853537914 None
0.0007251026803234345 None
0.0007133282166977083 None
0.0007054009235959999 None
0.0007171229681009652 None
0.0007086605103127501 None
0.0007619279094105562 None
0.0007293109036890235 None
0.0007196831816109351 None
0.0007088329353034388 None
0.0007133428223008093 None


  0%|          | 0/1000 [00:00<?, ?it/s]

0.009199231823901779 None
0.0030812653534717286 None
0.0031808846347645167 None
0.0030896390759006134 None
0.003104081974468812 None
0.003079531714266939 None
0.003088526735708203 None
0.0030904831913034117 None
0.0030774581448077274 None
0.0031101253549151255 None
0.0030761524030629215 None
Done!


In [12]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)
rng = np.random.default_rng(seed=42)

[distance_best_list, model_best_list] = loader(data_path("diluted_fitting_16layer.model"))
rank_start = model_best_list[-1].channel.rank

distance_best_list, model_best_list = fit_diluted_unitary(spectrum_target_list3, 
                                                          d=d, 
                                                          rank_start=rank_start, 
                                                          rank_step=1,
                                                          num_iter = 1000)

saver([distance_best_list, model_best_list], data_path("diluted_fitting_24layer.model"))

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

0.1440714338754331 None
0.11599898358034283 None
0.08805865631096427 None
0.0604716483699036 None
0.03342646739636095 None
0.013420228908767717 None
0.005878371024653105 None
0.005790152340618375 None
0.00579566694757544 None
0.005789830110670299 None
0.005830122998365071 None


  0%|          | 0/1000 [00:00<?, ?it/s]

0.00893389771812677 None
0.00684404021103127 None
0.006897863076171526 None
0.006826675702237325 None
0.0068149995004812836 None
0.006804422775949682 None
0.02313977668197024 None
0.006814849430422092 None
0.006805132034684727 None
0.006875592657177779 None
0.006831082820413225 None


...

0.0054215980269179745 None
0.004226531782496521 None
0.004230678300237463 None
0.004217959533067379 None
0.00417971895903266 None
0.004251439789013987 None
0.004197836592777096 None
0.004214329636036071 None
0.004189826534414752 None
0.004189795349659175 None
0.004236062520660493 None
Done!


  0%|          | 0/1000 [00:00<?, ?it/s]

0.07449047874723115 None
0.04690913969641585 None
0.01955107463443371 None
0.0018877318542723117 None
0.0017656526377063703 None
0.0017591467781654544 None
0.0017704312739952879 None
0.0017741364772519835 None
0.001787469932889678 None
0.0017756898957965973 None
0.0017793147278143558 None


  0%|          | 0/1000 [00:00<?, ?it/s]

0.006403677269740585 None
0.0016785585825552097 None
0.01745073413876011 None
0.0016675420151794703 None
0.01741388412255538 None
0.017408487538988316 None
0.0017209383290062899 None
0.017452160253697228 None
0.017470191055937074 None
0.01739772489193681 None
0.017400881179472784 None
Done!


In [13]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)
rng = np.random.default_rng(seed=42)

[distance_best_list, model_best_list] = loader(data_path("diluted_fitting_24layer.model"))
rank_start = model_best_list[-1].channel.rank

distance_best_list, model_best_list = fit_diluted_unitary(spectrum_target_list4, 
                                                          d=d, 
                                                          rank_start=rank_start, 
                                                          rank_step=1,
                                                          num_iter = 1000)

saver([distance_best_list, model_best_list], data_path("diluted_fitting_32layer.model"))

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

0.27973486457601465 None
0.2516624142809244 None
0.22372208701154578 None
0.1961350790704851 None
0.16908989809694247 None
0.14278309260127076 None
0.11734471960366481 None
0.09283565683944883 None
0.06923180316491784 None
0.04661768307228525 None
0.02493611801933672 None


  0%|          | 0/1000 [00:00<?, ?it/s]

0.03120998061273629 None
0.028723230482348726 None
0.02127450599403434 None
0.004949260846352612 None
0.021276655270743038 None
0.004936974610250118 None
0.004958296492640303 None
0.021272730683885455 None
0.02125989540089761 None
0.004952304083799529 None
0.004953203275312433 None


...

  0%|          | 0/1000 [00:00<?, ?it/s]

0.06775987537629488 None
0.04559923673823087 None
0.01135901026402068 None
0.022390205060235695 None
0.03781763107793493 None
0.022379058686628067 None
0.022410867536961778 None
0.022391630812166174 None
0.006073478218473021 None
0.022373953642041602 None
0.022370165928559206 None


  0%|          | 0/1000 [00:00<?, ?it/s]

0.06837440900450374 None
0.046627131685330656 None
0.02982229392959213 None
0.024693134032222758 None
0.008352108332158426 None
0.008379057474056793 None
0.02467854820005855 None
0.02472214181998139 None
0.024655971176555453 None


KeyboardInterrupt: 

## Four Qubits

In [38]:
n = 4
d = 2**n

[model1, model2, model3, model4, model5] = loader(data_path("belem_4qubit_repeated.model"))

spectrum1 = channel_spectrum(model1.channel, use_coords=True)
spectrum2 = channel_spectrum(model2.channel, use_coords=True) 
spectrum3 = channel_spectrum(model3.channel, use_coords=True)
spectrum4 = channel_spectrum(model4.channel, use_coords=True)
spectrum5 = channel_spectrum(model5.channel, use_coords=True)

In [39]:
distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum1, 
                                                      d = d, 
                                                      rank_start = 1, 
                                                      rank_step = 1,
                                                      num_iter = 1000)

model_best.optimizer = None
saver([distance_best, model_best], data_path("diluted_fitting_4qubit_8layer.model"))

  0%|          | 0/1000 [00:00<?, ?it/s]

0.2085285055390247 None
0.20772108047127816 None
0.20304760627128066 None
0.20304064670395125 None
0.2069356299367542 None
0.20304064668710342 None
0.2030406466871089 None
0.20693562993675918 None
0.2069356299367567 None
0.20693562993676387 None
0.2069356299367602 None


  0%|          | 0/1000 [00:00<?, ?it/s]

0.11264331762034509 None
0.0980024674459395 None
0.08987219399363602 None
0.08840771668118627 None
0.08463397391653817 None
0.08439493711056795 None
0.08047951703712536 None
0.08047595936629584 None
0.0843816241188914 None
0.08048799919145525 None
0.08048236400424513 None


  0%|          | 0/1000 [00:00<?, ?it/s]

...

0.07544832727050488 None
0.047983374209919605 None
0.020495751950615575 None
0.00266660494267037 None
0.002519459108687905 None
0.0024549374849748124 None
0.00248198623793877 None
0.0024772515921205646 None
0.002486610360585952 None
0.002481699596648453 None
0.002464453382341284 None


  0%|          | 0/1000 [00:00<?, ?it/s]

0.08038972484698108 None
0.04932932829120148 None
0.02583176606751496 None
0.0016415419940773344 None
0.00539654763815662 None
0.005381804959246181 None
0.009250882757283517 None
0.005411452103680733 None
0.0015054436387361438 None
0.005386699433998443 None
0.0015384608055065562 None


  0%|          | 0/1000 [00:00<?, ?it/s]

0.07564137349156974 None
0.04860920626157788 None
0.02161592917454029 None
0.0018700385480870058 None
0.001617774365076723 None
0.0016150190354173955 None
0.0016229007402945677 None
0.0016310892147336394 None
0.0016152285462323463 None
0.0016117318143597462 None
0.0016449867136785795 None
Done!


In [40]:
distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum2, 
                                                      d = d, 
                                                      rank_start = 1, 
                                                      rank_step = 1,
                                                      num_iter = 1000)

model_best.optimizer = None
saver([distance_best, model_best], data_path("diluted_fitting_4qubit_16layer.model"))

  0%|          | 0/1000 [00:00<?, ?it/s]

0.3529513921426881 None
0.345806609160031 None
0.3440198959077034 None
0.34376247216784356 None
0.3437408139234092 None
0.3397908991599398 None
0.34373910900830096 None
0.33979079231476966 None
0.3397907921962647 None
0.34373910447705985 None
0.34373910447704487 None


  0%|          | 0/1000 [00:00<?, ?it/s]

0.250967453470484 None
0.24350532361898186 None
0.23184137963658052 None
0.2247380449891482 None
0.22863315635506792 None
0.22863315635120013 None
0.2324753629364637 None
0.22863315635106185 None
0.22473803692417552 None
0.22863315635109657 None
0.2286331563511019 None


  0%|          | 0/1000 [00:00<?, ?it/s]

...

0.22813497407010586 None
0.20516523373941534 None
0.178303863784635 None
0.1516834018602287 None
0.12153802683019785 None
0.09582957478698785 None
0.07065075261252601 None
0.05357838714627559 None
0.04011512454082793 None
0.027357670603929515 None
0.01947512505756322 None


  0%|          | 0/1000 [00:00<?, ?it/s]

0.22846413978072966 None
0.19765559477213673 None
0.17483410438178562 None
0.14834916218940206 None
0.12230124385554668 None
0.09269109556431822 None
0.06767418285884577 None
0.050173549896082296 None
0.03703190008798793 None
0.024663674944777513 None
0.009173463758337967 None


  0%|          | 0/1000 [00:00<?, ?it/s]

0.2305143131514942 None
0.2036806260601801 None
0.17689531040916348 None
0.1503958122956705 None
0.12034694118531288 None
0.09476613804492098 None
0.0738847379425756 None
0.05213050372418569 None
0.0385508245308883 None
0.025591005072525992 None
0.013467920069588461 None
Done!


In [41]:
distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum3, 
                                                      d = d, 
                                                      rank_start = 1, 
                                                      rank_step = 1,
                                                      num_iter = 1000)

model_best.optimizer = None
saver([distance_best, model_best], data_path("diluted_fitting_4qubit_24layer.model"))

  0%|          | 0/1000 [00:00<?, ?it/s]

0.4539175078479235 None
0.45174514281701017 None
0.45165340280746924 None
0.45165339931227494 None
0.4477052567735745 None
0.4477052567735713 None
0.4516533993122748 None
0.4516533993122908 None
0.4516533993122783 None
0.45165339931229403 None
0.45554839402846736 None


  0%|          | 0/1000 [00:00<?, ?it/s]

0.3589258361692094 None
0.3517529726960299 None
0.34024372616421006 None
0.3424727992935695 None
0.3273484740336213 None
0.3290353957370658 None
0.3357251906420423 None
0.3279189447674703 None
0.3318138838559316 None
0.3318138838560613 None
0.3318138838589909 None


  0%|          | 0/1000 [00:00<?, ?it/s]

...

0.3480722205412848 None
0.32108901077602314 None
0.29405431448806957 None
0.26719776739859497 None
0.24471988597183328 None
0.21876276115296103 None
0.18945799783358508 None
0.16077988856377814 None
0.15180019784759524 None
0.1411627135295701 None
0.1351582658926172 None


  0%|          | 0/1000 [00:00<?, ?it/s]

0.34531633584905347 None
0.3183544425941973 None
0.2954230372429991 None
0.272700966342984 None
0.24246221160927361 None
0.21653485758437707 None
0.19113510664970282 None
0.1703523124677783 None
0.14344893569675055 None
0.1405952565113639 None
0.12673698236536343 None


  0%|          | 0/1000 [00:00<?, ?it/s]

0.3488273007525703 None
0.3179745422829954 None
0.2989845946623197 None
0.2685571310672928 None
0.24246179262153295 None
0.21298132258259345 None
0.19183449438834183 None
0.1674903451589837 None
0.14852292633569716 None
0.12928814489619678 None
0.12693192516140656 None
Done!


In [42]:
distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum4, 
                                                      d = d, 
                                                      rank_start = 1, 
                                                      rank_step = 1,
                                                      num_iter = 1000)

model_best.optimizer = None
saver([distance_best, model_best], data_path("diluted_fitting_4qubit_32layer.model"))

  0%|          | 0/1000 [00:00<?, ?it/s]

0.5259592882768112 None
0.5202041201413758 None
0.5173054131193121 None
0.5169367575737605 None
0.5168865588654987 None
0.5207765450952775 None
0.520776127212733 None
0.5168811397211148 None
0.5168811385784982 None
0.5207760999127549 None
0.5168811385435712 None


  0%|          | 0/1000 [00:00<?, ?it/s]

0.4204554912464412 None
0.4044448859764608 None
0.39963247750444386 None
0.39373261519852065 None
0.3911295415453419 None
0.38673015858348586 None
0.40241641247839655 None
0.3869430399798697 None
0.39478617723455706 None
0.38694303997976276 None
0.3908912103302758 None


  0%|          | 0/1000 [00:00<?, ?it/s]

...

0.4220555949085249 None
0.39882295651692995 None
0.3716480964717377 None
0.3407258417220768 None
0.31397603898267656 None
0.29156193258022384 None
0.26966354620498845 None
0.22846234268504798 None
0.21572284178919632 None
0.19696254035220376 None
0.19610425083128294 None


  0%|          | 0/1000 [00:00<?, ?it/s]

0.42371375658336974 None
0.4005474827056314 None
0.3695671536295553 None
0.3426040343721518 None
0.3120518211532069 None
0.29383399377973707 None
0.2646585792065765 None
0.2401839927086912 None
0.20877060227960556 None
0.197589430397304 None
0.18851782005592663 None


  0%|          | 0/1000 [00:00<?, ?it/s]

0.4254463010042312 None
0.3985748982126133 None
0.37180755290226525 None
0.34926012505020493 None
0.31943645214737043 None
0.29016032597232955 None
0.265433930088053 None
0.24547062709180908 None
0.22207846502112064 None
0.19587896235529678 None
0.1910411242146915 None
Done!
